# Scapy Hands-on at #GreHack16

## Description

Scapy (http://www.secdev.org/projects/scapy, https://github.com/secdev/scapy & http://scapy.readthedocs.io/en/latest/) is a powerful Python-based interactive packet manipulation program and library. It can be used to forge or decode packets for a wide number of protocols, send them on the wire, capture them, match requests and replies, and much more.

Each of them is different and gives you the opportunity to enhance your Scapy skills by manipulating yourself and watching a potential solution.


## Trophies

- :trophy: #1 - [Manipulating Packets](trophies/manipulating_packets.md)
- :trophy: #2 - [Network Interactions](trophies/network_interactions.md)
- :trophy: #3 - [Intercepting And Modifying Packets](trophies/intercepting_and_modifying.md)
- :trophy: #4 - [IPv6 Reconnaissance](trophies/ipv6_reconnaissance.md)
- :trophy: #5 - [Visualizations](trophies/visualizations.md)
- :trophy: #6 - [Fun With X.509 Certificates](trophies/fun_with_x509.md)
- :trophy: #7 - [Playing With TLS](trophies/playing_with_tls.md)
- :trophy: #8 - [Adding a New Protocol](trophies/new_protocol.md)
- :trophy: #9 - [Answering Machines](trophies/answering_machines.md)
- :trophy: #10 - [Pipes Introduction](trophies/pipes_introduction.md)

# :trophy: #1 - [Manipulating Packets](trophies/manipulating_packets.md)

# Manipulating packets

This trophy gives you the opportunity to perform simple Scapy packets manipulation.

## Tasks

**task #1**

- create a valid DNS query to `8.8.8.8`

In [15]:
from scapy.all import *
pkt=IP(dst="8.8.8.8")/UDP(sport=9999, dport=53)/DNS(qd=DNSQR(qname="github.com",qtype="A",qclass="IN"))
ans=sr1(pkt)

Begin emission:
Finished to send 1 packets.

Received 46 packets, got 1 answers, remaining 0 packets


- display its summary

In [16]:
ans.summary()

'IP / UDP / DNS Ans '

- access the source address computed by Scapy

In [17]:
ans[IP].src

'8.8.8.8'

- use the `sprintf()` method to display the source UDP port

In [18]:
ans.sprintf("The source UDP port = %UDP.sport%")

'The source UDP port = domain'

- access the layer before the DNS one

In [19]:
ans[DNS].underlayer

<UDP  sport=domain dport=9999 len=36 chksum=0x9278 |<DNS  id=0 qr=1 opcode=QUERY aa=0 tc=0 rd=0 ra=1 z=0 ad=0 cd=0 rcode=server-failure qdcount=1 ancount=0 nscount=0 arcount=0 qd=<DNSQR  qname='github.com.' qtype=A qclass=IN |> an=None ns=None ar=None |>>

- access the UDP layer

In [20]:
ans[UDP].len

36

**task #2**

- create an implicit packet that builds 5 explicit packets with 5 different TTL values

In [65]:
pkts=IP(dst="8.8.8.8",ttl=list(range(1,5)))/UDP(dport=53)/DNS(qd=DNSQR(qname="github.com",qtype="A",qclass="IN"))
pkts

<IP  frag=0 ttl=[1, 2, 3, 4] proto=udp dst=8.8.8.8 |<UDP  sport=domain dport=domain |<DNS  qd=<DNSQR  qname='github.com' qtype=A qclass=IN |> |>>>

- iterate over this packets

In [66]:
print(pkts[0].src,end=" -> ")
for pkt in pkts:
    ans1=sr1(pkt,verbose=0)
    print(ans1.src,end=" -> ")
#traceroute("8.8.8.8",l4=UDP(dport=53))

153.120.17.24 -> 153.120.17.2 -> 103.10.113.25 -> 103.10.113.9 -> 103.10.113.117 -> 

## Hints

- `ls()` list Scapy protocols names
- `ls(PROTO)` list the fields of the _PROTO_ protocol
- the `/` operator stacks protocols
- four Scapy protocol headers are required
- https://blogs.sans.org/pen-testing/files/2016/04/ScapyCheatSheet_v0.2.pdf

# :trophy: #2 - [Network Interactions](trophies/network_interactions.md)

# Network interactions

This trophy is about sending and receiving packets on the network.

## Tasks

**task #1**

- build a simple ICMP packet to 8.8.8.8

In [67]:
pkt = IP(dst="8.8.8.8")/ICMP()

- send it to the network

In [68]:
ans,uns = sr(pkt)

Begin emission:
Finished to send 1 packets.

Received 42 packets, got 1 answers, remaining 0 packets


- display the whole reply using `show()`

In [69]:
ans.show()

0000 IP / ICMP 153.120.17.24 > 8.8.8.8 echo-request 0 ==> IP / ICMP 8.8.8.8 > 153.120.17.24 echo-reply 0 / Padding


**task #2**

- build an implicit ICMP frame (i.e. layer 2)  with a TTL value from to 2 to 7

In [70]:
pkts = IP(dst="8.8.8.8",ttl=[2,3,4,5,6,7])/ICMP()

- send it to the network

In [71]:
ans,uns = sr(pkts)

Begin emission:
Finished to send 6 packets.

Received 54 packets, got 6 answers, remaining 0 packets


- use `nsummary()` to show replies

In [72]:
ans.nsummary()

0000 IP / ICMP 153.120.17.24 > 8.8.8.8 echo-request 0 ==> IP / ICMP 103.10.113.25 > 153.120.17.24 time-exceeded ttl-zero-during-transit / IPerror / ICMPerror
0001 IP / ICMP 153.120.17.24 > 8.8.8.8 echo-request 0 ==> IP / ICMP 103.10.113.101 > 153.120.17.24 time-exceeded ttl-zero-during-transit / IPerror / ICMPerror
0002 IP / ICMP 153.120.17.24 > 8.8.8.8 echo-request 0 ==> IP / ICMP 103.10.113.113 > 153.120.17.24 time-exceeded ttl-zero-during-transit / IPerror / ICMPerror
0003 IP / ICMP 153.120.17.24 > 8.8.8.8 echo-request 0 ==> IP / ICMP 157.17.131.33 > 153.120.17.24 time-exceeded ttl-zero-during-transit / IPerror / ICMPerror
0004 IP / ICMP 153.120.17.24 > 8.8.8.8 echo-request 0 ==> IP / ICMP 210.171.224.96 > 153.120.17.24 time-exceeded ttl-zero-during-transit / IPerror / ICMPerror
0005 IP / ICMP 153.120.17.24 > 8.8.8.8 echo-request 0 ==> IP / ICMP 108.170.242.161 > 153.120.17.24 time-exceeded ttl-zero-during-transit / IPerror / ICMPerror


- display the address of the third router on the path

In [79]:
req,resp = ans[2]
resp.src

'103.10.113.113'

- display the results as hex strings

In [82]:
ans.hexdump()

0000 08:32:39.831434 IP / ICMP 153.120.17.24 > 8.8.8.8 echo-request 0 ==> IP / ICMP 103.10.113.25 > 153.120.17.24 time-exceeded ttl-zero-during-transit / IPerror / ICMPerror
0000   45 00 00 38 DE 28 00 00  3F 01 1A E9 67 0A 71 19   E..8.(..?...g.q.
0010   99 78 11 18 0B 00 F4 FF  00 00 00 00 45 00 00 1C   .x..........E...
0020   00 01 00 00 01 01 FF 40  99 78 11 18 08 08 08 08   .......@.x......
0030   08 00 F7 FF 00 00 00 00                            ........
0001 08:32:39.832900 IP / ICMP 153.120.17.24 > 8.8.8.8 echo-request 0 ==> IP / ICMP 103.10.113.101 > 153.120.17.24 time-exceeded ttl-zero-during-transit / IPerror / ICMPerror
0000   45 00 00 38 00 00 00 00  FD 01 3A C5 67 0A 71 65   E..8......:.g.qe
0010   99 78 11 18 0B 00 F4 FF  00 00 00 00 45 00 00 1C   .x..........E...
0020   00 01 00 00 01 01 FF 40  99 78 11 18 08 08 08 08   .......@.x......
0030   08 00 F7 FF 00 00 00 00                            ........
0002 08:32:39.834206 IP / ICMP 153.120.17.24 > 8.8.8.8 echo-request

**task #3**

- send ICMP echo request forever using the `sr*()` function that send packets forever.

In [83]:
pkts = IP(dst="8.8.8.8")/ICMP()
srloop(pkt)
#- stop the function: C-c interrupt

RECV 1: IP / ICMP 8.8.8.8 > 153.120.17.24 echo-reply 0 / Padding
RECV 1: IP / ICMP 8.8.8.8 > 153.120.17.24 echo-reply 0 / Padding
 end... 
Sent 2 packets, received 2 packets. 100.0% hits.


(<Results: TCP:0 UDP:0 ICMP:2 Other:0>,
 <PacketList: TCP:0 UDP:0 ICMP:0 Other:0>)

- use the `prn` argument to display the source IP address of the replies

In [91]:
pkt = IP(dst="8.8.8.8")/ICMP()
result = srloop(pkt,prn=lambda pkts:print(pkts[1][IP].src))
#- stop the function: C-c interrupt

RECV 1: 8.8.8.8
None
RECV 1: 8.8.8.8
None
RECV 1: 8.8.8.8
None
RECV 1: 8.8.8.8
None
        
Sent 4 packets, received 4 packets. 100.0% hits.


**task #4**

- sniff 5 packets on port 443/tcp

In [94]:
sslpkts = sniff(lfilter=lambda pkt:TCP in pkt and pkt[TCP].dport == 443, count=5)
sslpkts.show()

0000 Ether / IP / TCP 210.149.252.70:48487 > 153.120.17.24:https A / Padding
0001 Ether / IP / TCP 210.149.252.70:48487 > 153.120.17.24:https A / Padding
0002 Ether / IP / TCP 210.149.252.70:48487 > 153.120.17.24:https A / Padding
0003 Ether / IP / TCP 210.149.252.70:48487 > 153.120.17.24:https PA / Raw
0004 Ether / IP / TCP 153.120.17.24:53867 > 172.217.26.99:https S


- display them in wireshark (without leaving Scapy)

In [ ]:
wireshark(sslpkts)

- write these 5 packets into `test.pcap`

In [ ]:
wrpcap("sslpkts.cap",sslpkts)

- read them back from the saved PCAP file into the pkts variable

In [ ]:
sslpkts_read = rdpcap("sslpkts.cap")

- use `+` to concatenate these packets

In [ ]:
w_sslpkts = sslpkts + sslpkts_read

## Hints

- `sr*()` functions send at layer 3
- `srp*()` functions send at layer 2
- `lsc()` list Scapy functions

# :trophy: #3 - Intercepting And Modifying Packets

# Intercepting and modifying packets

This trophy shows howto use Scapy with NFQUEUE in order to tag and modify
packets on the fly.

## NOTE
```
nfqueue library => netfilterqueue library
```

## Tasks

**task #1**

- edit the `nfq_trophy.py` template

In [ ]:
# Guillaume Valadon <guillaume@valadon.net>

"""Template to use Scapy with NFQUEUE"""

# Note: the following command must be used before launching this script:
# sudo iptables -A INPUT -s 8.8.8.8 -p udp --sport 53 -j NFQUEUE --queue-num 2807

# apt-get install libnfnetlink-dev
# apt-get install libnetfilter-queue-dev
# pip install NetfilterQueue

from scapy.all import *
from netfilterqueue import NetfilterQueue

def scapy_callback(pkt):
    # Get the data
    data = IP(pkt.get_payload())

    #pkt.set_payload()

    # Accept the packet
    pkt.accept()
    #pkt.drop()
    #pkt.repeat()

if __name__ == "__main__":
    # Get an NFQUEUE handler
    q = NetfilterQueue()

    # Set the callback
    q.bind(2807, scapy_callback)
    try:
        # Open the queue and start parsing packets
        q.run()
    except KeyboardInterrupt:
        print("")
    q.unbind()

- trigger a packet with `dig` to get the IP address of grehack.fr

In [ ]:
#dig grehack.fr @8.8.8.8

- check if it works as expected

In [ ]:
#check script running is stopped.



**task #2**


- parse the packet with Scapy

In [ ]:
# Guillaume Valadon <guillaume@valadon.net>

"""Template to use Scapy with NFQUEUE"""

# Note: the following command must be used before launching this script:
# sudo iptables -A INPUT -s 8.8.8.8 -p udp --sport 53 -j NFQUEUE --queue-num 2807

# apt-get install libnfnetlink-dev
# apt-get install libnetfilter-queue-dev
# pip install NetfilterQueue

from scapy.all import *
from netfilterqueue import NetfilterQueue

def scapy_callback(pkt):
    # Get the data
    data = IP(pkt.get_payload())

    data.show() # Show the parsed data!
    #pkt.set_payload()

    # Accept the packet
    pkt.accept()
    #pkt.drop()
    #pkt.repeat()

if __name__ == "__main__":
    # Get an NFQUEUE handler
    q = NetfilterQueue()

    # Set the callback
    q.bind(2807, scapy_callback)
    try:
        # Open the queue and start parsing packets
        q.run()
    except KeyboardInterrupt:
        print("")
    q.unbind()

- check if it contains as DNS header

In [ ]:
# Guillaume Valadon <guillaume@valadon.net>

"""Template to use Scapy with NFQUEUE"""

# Note: the following command must be used before launching this script:
# sudo iptables -A INPUT -s 8.8.8.8 -p udp --sport 53 -j NFQUEUE --queue-num 2807

# apt-get install libnfnetlink-dev
# apt-get install libnetfilter-queue-dev
# pip install NetfilterQueue

from scapy.all import *
from netfilterqueue import NetfilterQueue

def scapy_callback(pkt):
    # Get the data
    data = IP(pkt.get_payload())

    if DNS in data:
        print("DNS in packet.!")
    else:
        print("DNS not in a packet.")
    #pkt.set_payload()

    # Accept the packet
    pkt.accept()
    #pkt.drop()
    #pkt.repeat()

if __name__ == "__main__":
    # Get an NFQUEUE handler
    q = NetfilterQueue()

    # Set the callback
    q.bind(2807, scapy_callback)
    try:
        # Open the queue and start parsing packets
        q.run()
    except KeyboardInterrupt:
        print("")
    q.unbind()

- remove checksums and lengths

In [ ]:
# Guillaume Valadon <guillaume@valadon.net>

"""Template to use Scapy with NFQUEUE"""

# Note: the following command must be used before launching this script:
# sudo iptables -A INPUT -s 8.8.8.8 -p udp --sport 53 -j NFQUEUE --queue-num 2807

# apt-get install libnfnetlink-dev
# apt-get install libnetfilter-queue-dev
# pip install NetfilterQueue

from scapy.all import *
from netfilterqueue import NetfilterQueue

def scapy_callback(pkt):
    # Get the data
    data = IP(pkt.get_payload())

    del data.len
    del data.chksum

    pkt.set_payload(data.build())

    # Accept the packet
    pkt.accept()
    #pkt.drop()
    #pkt.repeat()

if __name__ == "__main__":
    # Get an NFQUEUE handler
    q = NetfilterQueue()

    # Set the callback
    q.bind(2807, scapy_callback)
    try:
        # Open the queue and start parsing packets
        q.run()
    except KeyboardInterrupt:
        print("")
    q.unbind()

- ~~use `set_verdict_modified()` to send the packet processed by Scapy~~

**task #3**

- iterate over the received DNS Resource Records

In [ ]:
# Guillaume Valadon <guillaume@valadon.net>

"""Template to use Scapy with NFQUEUE"""

# Note: the following command must be used before launching this script:
# sudo iptables -A INPUT -s 8.8.8.8 -p udp --sport 53 -j NFQUEUE --queue-num 2807

# apt-get install libnfnetlink-dev
# apt-get install libnetfilter-queue-dev
# pip install NetfilterQueue

from scapy.all import *
from netfilterqueue import NetfilterQueue

def scapy_callback(pkt):
    # Get the data
    data = IP(pkt.get_payload())

    if DNSRR in data:
        for i in range(data.ancount):
            dnsrr = data.an[i]
            rrname = dnsrr.rrname
            rrdata = dnsrr.rdata
            print(rrname,rrdata)

    #pkt.set_payload(data.build())

    # Accept the packet
    pkt.accept()
    #pkt.drop()
    #pkt.repeat()

if __name__ == "__main__":
    # Get an NFQUEUE handler
    q = NetfilterQueue()

    # Set the callback
    q.bind(2807, scapy_callback)
    try:
        # Open the queue and start parsing packets
        q.run()
    except KeyboardInterrupt:
        print("")
    q.unbind()

- identify the grehack.fr address

- change it to 127.0.0.1

In [ ]:
# Guillaume Valadon <guillaume@valadon.net>

"""Template to use Scapy with NFQUEUE"""

# Note: the following command must be used before launching this script:
# sudo iptables -A INPUT -s 8.8.8.8 -p udp --sport 53 -j NFQUEUE --queue-num 2807

# apt-get install libnfnetlink-dev
# apt-get install libnetfilter-queue-dev
# pip install NetfilterQueue

from scapy.all import *
from netfilterqueue import NetfilterQueue

def scapy_callback(pkt):
    # Get the data
    data = IP(pkt.get_payload())

    if DNSRR in data:
        for i in range(data.ancount):
            if("grehack.fr." in data.an[i].rrname):
                del data[IP].len
                del data[IP].chksum
                del data[UDP].len
                del data[UDP].chksum
                data.an[i].rdata = "127.0.0.1"

    pkt.set_payload(data.build())

    # Accept the packet
    pkt.accept()
    #pkt.drop()
    #pkt.repeat()

if __name__ == "__main__":
    # Get an NFQUEUE handler
    q = NetfilterQueue()

    # Set the callback
    q.bind(2807, scapy_callback)
    try:
        # Open the queue and start parsing packets
        q.run()
    except KeyboardInterrupt:
        print("")
    q.unbind()



## Hints

- on Debian, install the `python-nfqueue` module
- DNS is an *old* Scapy protocol, parsing then building might not give the same
  packet: you need to remove lengths

# :trophy: #4 - [IPv6 Reconnaissance](trophies/ipv6_reconnaissance.md)

# IPv6 reconnaissance

This trophy provides you some simple tricks to perform IPv6 reconnaissances with
Scapy.

## Tasks

**task #1**

- build a simple IPv6 packet with a Router Solicitation (RS) packet on top
- send it with sr1
- do you have an answer ?

In [15]:
from scapy.all import *
pkt = IPv6()/ICMPv6ND_RS()

In [16]:
sr1(pkt)

Begin emission:
Finished to send 1 packets.

Received 62 packets, got 0 answers, remaining 1 packets


**task #2**

- build a simple IPv6 `ping6` packet to ff0::1 (i.e. all nodes)

In [17]:
pkt = IPv6(dst="ff0::1")/ICMPv6EchoRequest()

In [ ]:
sr1(pkt)

- set the `conf.checkIPsrc` variable to False
- send it with sr1
- do you have an answer ?

In [19]:
conf.checkIPsrc = False

In [ ]:
sr1(pkt)

**task #3**

- use `srloop` to send the previously built message, using the `multi` argument
- do you have many answers ?
- stop it
- add a `prn` argument to display all received IPv6 source addresses

In [ ]:
srloop(IPv6(dst="ff0::1")/ICMPv6EchoRequest(), multi=True)

In [ ]:
srloop(IPv6(dst="ff0::1")/ICMPv6EchoRequest(),multi=True, prn=lambda reqresp:print(reqresp[1].src)


## Hints

- all IPv6 related layers start with **IPv6** or **ICMP6**: use TAB !

# trophy: #5 - [Visualizations](trophies/visualizations.md)

# Visualizations

This trophy shows you some Scapy visualization tools.

## Tasks

**task #1**

- create the packet of your choice

In [24]:
pkt = Ether()/IP()/TCP()

- use `str` and `hexdump` to show the build packet values

In [25]:
str(pkt)

"b'\\xff\\xff\\xff\\xff\\xff\\xff\\x00\\x00\\x00\\x00\\x00\\x00\\x08\\x00E\\x00\\x00(\\x00\\x01\\x00\\x00@\\x06|\\xcd\\x7f\\x00\\x00\\x01\\x7f\\x00\\x00\\x01\\x00\\x14\\x00P\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00P\\x02 \\x00\\x91|\\x00\\x00'"

In [26]:
hexdump(pkt)

0000   FF FF FF FF FF FF 00 00  00 00 00 00 08 00 45 00   ..............E.
0010   00 28 00 01 00 00 40 06  7C CD 7F 00 00 01 7F 00   .(....@.|.......
0020   00 01 00 14 00 50 00 00  00 00 00 00 00 00 50 02   .....P........P.
0030   20 00 91 7C 00 00                                   ..|..


- call `show` and `show2` on the packet and notice the differences

In [28]:
pkt.show()

###[ Ethernet ]###
  dst       = ff:ff:ff:ff:ff:ff
  src       = 00:00:00:00:00:00
  type      = 0x800
###[ IP ]###
     version   = 4
     ihl       = None
     tos       = 0x0
     len       = None
     id        = 1
     flags     = 
     frag      = 0
     ttl       = 64
     proto     = tcp
     chksum    = None
     src       = 127.0.0.1
     dst       = 127.0.0.1
     \options   \
###[ TCP ]###
        sport     = ftp_data
        dport     = http
        seq       = 0
        ack       = 0
        dataofs   = None
        reserved  = 0
        flags     = S
        window    = 8192
        chksum    = None
        urgptr    = 0
        options   = {}


In [29]:
pkt.show2()

###[ Ethernet ]###
  dst       = ff:ff:ff:ff:ff:ff
  src       = 00:00:00:00:00:00
  type      = 0x800
###[ IP ]###
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 40
     id        = 1
     flags     = 
     frag      = 0
     ttl       = 64
     proto     = tcp
     chksum    = 0x7ccd
     src       = 127.0.0.1
     dst       = 127.0.0.1
     \options   \
###[ TCP ]###
        sport     = ftp_data
        dport     = http
        seq       = 0
        ack       = 0
        dataofs   = 5
        reserved  = 0
        flags     = S
        window    = 8192
        chksum    = 0x917c
        urgptr    = 0
        options   = {}


- call `pdfdump` or `canvas_dump`

In [31]:
pkt.pdfdump();pkt.canvas_dump();

<bound method Packet.pdfdump of <Ether  type=0x800 |<IP  frag=0 proto=tcp |<TCP  |>>>>

**task #2**

- use `srloop` to send 100 packets to `8.8.8.8` and `8.8.4.4`

In [6]:
req,resp = srloop(IP(dst=["8.8.8.8","8.8.4.4"])/ICMP(),count=5,verbose=0)

 end...

- call `multiplot` on the resulting packets list and plot IP id

In [27]:
#%matplotlib inline
#from scapy.all import *
#req,resp = srloop(IP(dst=["8.8.8.8","8.8.4.4"])/ICMP(),count=5,verbose=0)
#resp.multiplot(lambda pkt:pkt.id)

**task #3**

- use the `traceroute` function to get the path to two different targets

In [23]:
from scapy.all import *
traceroute(["www.google.co.jp","www.yahoo.co.jp"],l4=TCP(dport=80))

Begin emission:
Finished to send 60 packets.

Received 73 packets, got 28 answers, remaining 32 packets
   172.217.26.99:tcp80 183.79.249.252:tcp80 
1  153.120.17.2    11  153.120.17.2    11   
2  103.10.113.25   11  103.10.113.25   11   
3  103.10.113.93   11  103.10.113.93   11   
4  103.10.113.109  11  103.10.113.117  11   
5  157.17.131.33   11  157.17.131.37   11   
6  210.171.224.96  11  157.17.130.113  11   
7  108.170.242.130 11  157.17.130.130  11   
8  66.249.94.172   11  61.206.157.74   11   
9  209.85.252.142  11  124.83.228.197  11   
10 108.170.243.129 11  183.79.224.206  11   
11 108.170.235.141 11  183.79.249.252  SA   
12 172.217.26.99   SA  -                    
13 172.217.26.99   SA  -                    
14 172.217.26.99   SA  -                    
15 172.217.26.99   SA  -                    
16 172.217.26.99   SA  -                    
17 172.217.26.99   SA  -                    


(<Traceroute: TCP:7 UDP:0 ICMP:21 Other:0>,
 <Unanswered: TCP:32 UDP:0 ICMP:0 Other:0>)

- try `world_trace` and `trace3D`

In [29]:
#traced = world_trace(["8.8.8.8","8.8.4.4"])
#traced.trace3D()
#not test

## Hints

- check the help of `multiplot` and `traceroute`

# :trophy: #6 - [Fun With X.509 Certificates](trophies/fun_with_x509.md)


# Fun with X.509 certificates

Scapy is able to natively parse ASN.1. Internally, this feature is used for SNMP
as well as X.509 certificate parsing.

This trophy shows you some cool things that you can do.

## Tasks

**task #1**

- retrieve `grehack.fr` X.509 certificate as DER

In [42]:
%%sh
echo "Q" | openssl s_client -connect grehack.fr:443 -CApath /etc/ssl/certs/ | openssl x509 -inform PEM -text -outform DER -out grehack.fr.der

Certificate:
    Data:
        Version: 3 (0x2)
        Serial Number:
            03:70:61:0b:84:86:ea:43:20:f9:61:72:ca:87:4f:bb:d5:1a
    Signature Algorithm: sha256WithRSAEncryption
        Issuer: C=US, O=Let's Encrypt, CN=Let's Encrypt Authority X3
        Validity
            Not Before: Mar 26 03:47:00 2017 GMT
            Not After : Jun 24 03:47:00 2017 GMT
        Subject: CN=grehack.fr
        Subject Public Key Info:
            Public Key Algorithm: rsaEncryption
                Public-Key: (4096 bit)
                Modulus:
                    00:b6:68:2a:7e:59:9d:fd:84:f7:41:0b:ea:9b:9b:
                    0e:19:9a:06:f4:7b:a8:76:e3:56:a6:3e:8b:43:9b:
                    91:da:d4:85:cb:bf:09:ed:d1:c5:26:ce:59:85:80:
                    4c:ed:fc:f5:bb:d9:f0:7e:76:f3:39:9a:0e:82:d8:
                    26:66:c3:86:c7:e3:d9:0b:bd:29:a5:e1:c1:6d:7b:
                    35:63:6c:28:41:94:de:93:68:77:a0:26:4c:00:35:
                    e7:93:56:78:9d:9a:d0:98:c1:f7:b8:e3:25

depth=2 O = Digital Signature Trust Co., CN = DST Root CA X3
verify return:1
depth=1 C = US, O = Let's Encrypt, CN = Let's Encrypt Authority X3
verify return:1
depth=0 CN = grehack.fr
verify return:1
DONE


- use `der2pem()` to convert the certificate to PEM
- check the result with `openssl x509`

In [37]:
#from scapy.all import *
#der2pem("grehack.fr.der")

In [43]:
%%sh
echo "Q" | openssl s_client -connect grehack.fr:443 -CApath /etc/ssl/certs/ | openssl x509 -inform PEM -text

Certificate:
    Data:
        Version: 3 (0x2)
        Serial Number:
            03:70:61:0b:84:86:ea:43:20:f9:61:72:ca:87:4f:bb:d5:1a
    Signature Algorithm: sha256WithRSAEncryption
        Issuer: C=US, O=Let's Encrypt, CN=Let's Encrypt Authority X3
        Validity
            Not Before: Mar 26 03:47:00 2017 GMT
            Not After : Jun 24 03:47:00 2017 GMT
        Subject: CN=grehack.fr
        Subject Public Key Info:
            Public Key Algorithm: rsaEncryption
                Public-Key: (4096 bit)
                Modulus:
                    00:b6:68:2a:7e:59:9d:fd:84:f7:41:0b:ea:9b:9b:
                    0e:19:9a:06:f4:7b:a8:76:e3:56:a6:3e:8b:43:9b:
                    91:da:d4:85:cb:bf:09:ed:d1:c5:26:ce:59:85:80:
                    4c:ed:fc:f5:bb:d9:f0:7e:76:f3:39:9a:0e:82:d8:
                    26:66:c3:86:c7:e3:d9:0b:bd:29:a5:e1:c1:6d:7b:
                    35:63:6c:28:41:94:de:93:68:77:a0:26:4c:00:35:
                    e7:93:56:78:9d:9a:d0:98:c1:f7:b8:e3:25

depth=2 O = Digital Signature Trust Co., CN = DST Root CA X3
verify return:1
depth=1 C = US, O = Let's Encrypt, CN = Let's Encrypt Authority X3
verify return:1
depth=0 CN = grehack.fr
verify return:1
DONE


In [49]:
pem_s= b"""MIIGRjCCBS6gAwIBAgISA3BhC4SG6kMg+WFyyodPu9UaMA0GCSqGSIb3DQEBCwUA
MEoxCzAJBgNVBAYTAlVTMRYwFAYDVQQKEw1MZXQncyBFbmNyeXB0MSMwIQYDVQQD
ExpMZXQncyBFbmNyeXB0IEF1dGhvcml0eSBYMzAeFw0xNzAzMjYwMzQ3MDBaFw0x
NzA2MjQwMzQ3MDBaMBUxEzARBgNVBAMTCmdyZWhhY2suZnIwggIiMA0GCSqGSIb3
DQEBAQUAA4ICDwAwggIKAoICAQC2aCp+WZ39hPdBC+qbmw4Zmgb0e6h241amPotD
m5Ha1IXLvwnt0cUmzlmFgEzt/PW72fB+dvM5mg6C2CZmw4bH49kLvSml4cFtezVj
bChBlN6TaHegJkwANeeTVnidmtCYwfe44yV0qrspWKLwgNJk79GVrKko0tukvKYS
Qxx0G2MWcxens3EjeBkHA5nS3Hdh8kvxBD+4viFpJ81DTihsoROMpOBRZgjorPsU
LJGzGnje2kuW9rGUwfa+O15flTJQMiDZRUuj73tYU6biia/DkxTy42aS0tI8M0uE
THF3FcH0X5Npb7p+twyGwGh0X4+4+QNTOT/VSty1kERuTlyMAE8NOIeFCmnDIvkl
H2xguQv1t4KB8iBeWUktqe/Ohu2PAGaQR4AbgHO9WAuA2mGcNhTftLVmoTs+RRx0
rqdsAonFI4L3VLG4pBgZe6qV6bKhU0k6yOBokO2ka3ob6Cyp6IT9YnEiym5QuuM2
fa7LkaPWUmdSzKfZ19qSh0s+kKP0k/f7aSzw4jjNbwDOSV+M/hNQ6XuBkr0ZYkds
wHIL7QWBtKMAleA+rgFTO4JrvVGxC3/bISmRA6VT2qPENONEU6xIvd2s8C4S+oIK
xWMaaI25DYSKh3srUtHwbDrLcGn8wp+pT9pQagqytgGRtLzmio+K23sfERZycn/N
KFAmUQIDAQABo4ICWTCCAlUwDgYDVR0PAQH/BAQDAgWgMB0GA1UdJQQWMBQGCCsG
AQUFBwMBBggrBgEFBQcDAjAMBgNVHRMBAf8EAjAAMB0GA1UdDgQWBBS251k4s+kb
Hvd2cA/hA08gkr87kDAfBgNVHSMEGDAWgBSoSmpjBH3duubRObemRWXv86jsoTBw
BggrBgEFBQcBAQRkMGIwLwYIKwYBBQUHMAGGI2h0dHA6Ly9vY3NwLmludC14My5s
ZXRzZW5jcnlwdC5vcmcvMC8GCCsGAQUFBzAChiNodHRwOi8vY2VydC5pbnQteDMu
bGV0c2VuY3J5cHQub3JnLzBjBgNVHREEXDBaggpncmVoYWNrLmZyggxncmVoYWNr
LmluZm+CC2dyZWhhY2submV0gg53d3cuZ3JlaGFjay5mcoIQd3d3LmdyZWhhY2su
aW5mb4IPd3d3LmdyZWhhY2submV0MIH+BgNVHSAEgfYwgfMwCAYGZ4EMAQIBMIHm
BgsrBgEEAYLfEwEBATCB1jAmBggrBgEFBQcCARYaaHR0cDovL2Nwcy5sZXRzZW5j
cnlwdC5vcmcwgasGCCsGAQUFBwICMIGeDIGbVGhpcyBDZXJ0aWZpY2F0ZSBtYXkg
b25seSBiZSByZWxpZWQgdXBvbiBieSBSZWx5aW5nIFBhcnRpZXMgYW5kIG9ubHkg
aW4gYWNjb3JkYW5jZSB3aXRoIHRoZSBDZXJ0aWZpY2F0ZSBQb2xpY3kgZm91bmQg
YXQgaHR0cHM6Ly9sZXRzZW5jcnlwdC5vcmcvcmVwb3NpdG9yeS8wDQYJKoZIhvcN
AQELBQADggEBABedrp/2tO+z32YXtmVCZlmcmrQqj2/hwRSZ2RzHVrM7ovFUr80j
GN5hVjxmB0C/GFYNscUCKvuF6KMoRls9wOyRDm+q94NgSGPIDqWztxEixOyqeWC8
19ITyqrH2u2cfQHU20iRP4jXa8hDosMuRut2X/tu63/ar1te8ICc8UEWGlmflbvh
QeslCX26D6ZJ5FfolVhkMEFGrHtrt02jBDPE+t5Da4BeBYLoyHQWo82QZVwxuPxm
8wUAVJSDcQwA0TU+oGXMtRU8UnFCu7TF+pInUITrLCJjfK5gJb83TrECklcdKeSj
JFsZ+xtVt1/r4x7ZWLGxV39wMT1vrEFUE9o="""

import base64
pem = base64.decodebytes(pem_s)

- parse its content with the `X509_Cert` class

In [51]:
from scapy.all import *
cert = X509Cert(pem)

<UNIVERSAL> <class 'scapy.asn1.asn1.ASN1_Class_metaclass'>
<UNIVERSAL> <class 'scapy.asn1.asn1.ASN1_Class_metaclass'>
<UNIVERSAL> <class 'scapy.asn1.asn1.ASN1_Class_metaclass'>
<UNIVERSAL> <class 'scapy.asn1.asn1.ASN1_Class_metaclass'>
<UNIVERSAL> <class 'scapy.asn1.asn1.ASN1_Class_metaclass'>
<UNIVERSAL> <class 'scapy.asn1.asn1.ASN1_Class_metaclass'>
<UNIVERSAL> <class 'scapy.asn1.asn1.ASN1_Class_metaclass'>
<UNIVERSAL> <class 'scapy.asn1.asn1.ASN1_Class_metaclass'>
<UNIVERSAL> <class 'scapy.asn1.asn1.ASN1_Class_metaclass'>
<UNIVERSAL> <class 'scapy.asn1.asn1.ASN1_Class_metaclass'>
<UNIVERSAL> <class 'scapy.asn1.asn1.ASN1_Class_metaclass'>
<UNIVERSAL> <class 'scapy.asn1.asn1.ASN1_Class_metaclass'>
<UNIVERSAL> <class 'scapy.asn1.asn1.ASN1_Class_metaclass'>
<UNIVERSAL> <class 'scapy.asn1.asn1.ASN1_Class_metaclass'>
<UNIVERSAL> <class 'scapy.asn1.asn1.ASN1_Class_metaclass'>
<UNIVERSAL> <class 'scapy.asn1.asn1.ASN1_Class_metaclass'>
<UNIVERSAL> <class 'scapy.asn1.asn1.ASN1_Class_metaclass

- display the signature algorithm, and the subject name

In [81]:
cert.sign_algo #sha256WithRSAEncryption
cert.subject
cert.display()

###[ X509Cert ]###
  version   = <ASN1_INTEGER[2]>
  sn        = <ASN1_INTEGER[299577477809249570675521252408705060427034]>
  sign_algo = <ASN1_OID[b'.1.2.840.113549.1.1.11']>
  sa_value  = <ASN1_NULL[0]>
  \issuer    \
   |###[ X509RDN ]###
   |  oid       = <ASN1_OID[b'.2.5.4.6']>
   |  value     = <ASN1_PRINTABLE_STRING[b'US']>
   |###[ X509RDN ]###
   |  oid       = <ASN1_OID[b'.2.5.4.10']>
   |  value     = <ASN1_PRINTABLE_STRING[b"Let's Encrypt"]>
   |###[ X509RDN ]###
   |  oid       = <ASN1_OID[b'.2.5.4.3']>
   |  value     = <ASN1_PRINTABLE_STRING[b"Let's Encrypt Authority X3"]>
  not_before= <ASN1_UTC_TIME[b'170326034700Z']>
  not_after = <ASN1_UTC_TIME[b'170624034700Z']>
  \subject   \
   |###[ X509RDN ]###
   |  oid       = <ASN1_OID[b'.2.5.4.3']>
   |  value     = <ASN1_PRINTABLE_STRING[b'grehack.fr']>
  pubkey_algo= <ASN1_OID[b'.1.2.840.113549.1.1.1']>
  pk_value  = <ASN1_NULL[0]>
  pubkey    = <ASN1_BIT_STRING[b'\x000\x82\x02\n\x02\x82\x02\x01\x00\xb6h*~Y\x9d\xfd\x84\xf7

- find the name of the OCSP server

In [74]:
cert[10].val

<ASN1_SEQUENCE[[<ASN1_OID[b'.1.3.6.1.5.5.7.1.1']>, <ASN1_STRING[b'0b0/\x06\x08+\x06\x01\x05\x05\x070\x01\x86#http://ocsp.int-x3.letsencrypt.org/0/\x06\x08+\x06\x01\x05\x05\x070\x02\x86#http://cert.int-x3.letsencrypt.org/']>]]>

**task #2**

- use the `Cert` object to load the certificate from its filename

In [ ]:
cert = Cert("grehack.fr.der")

- display the number of days until expiration

In [ ]:
# cert.not_after

- is it a self signed certificate ?

In [ ]:
# cert.ASN1_root

**task #3**

- generate an RSA private key using `openssl genrsa`
- load it in Scapy with `PrivKey`
- assign your birthday to the certificate serial number
- use the `resignCert` method to sign the certificate with your own signature
- check the result with `openssl x509`
- verify the signature using the `PrivKey` `verify` method

## Hints

- X.509 certificate can be downloaded from a web browser status bar
- use the `obj` argument of `der2pem()` with the 'CERTIFICATE' type
- verify the new signature, set the `verify()` `h` and `t` arguments to sha256 and pkcs

# :trophy: #7 - [Playing With TLS](trophies/playing_with_tls.md)


# Playing with TLS

The pending [PR #294](playing_with_tls.md) brings a native TLS support to Scapy.

This trophy shows you some cool things that you can do.

## Tasks

**task #1**

- clone a new Scapy git repository from https://github.com/mtury/scapy.git, and
  swith to the tl2 branch

- :trophy: #7 - [Playing With TLS](trophies/playing_with_tls.md)
- :trophy: #8 - [Adding a New Protocol](trophies/new_protocol.md)
- :trophy: #9 - [Answering Machines](trophies/answering_machines.md)
- :trophy: #10 - [Pipes Introduction](trophies/pipes_introduction.md)